# SAR Change Detection Example (Jan 7/2021)

In [ ]:
# Some dependencies that might be needed
#! pip install sentinelsat
#! pip install geopandas
#! pip install folium
# CATALYST algorithms can be imported with 'import pci' once CATALYST Professional is installed and licensed 

In [ ]:
#display the AOI on a basemap

#define a basemap
import folium
m = folium.Map(
    zoom_start=10,
    #Imagery Map
    # tiles='https://server.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    
    #Street Map
    # tiles='https://server.arcgisonline.com/arcgis/rest/services/World_Street_Map/MapServer/tile/{z}/{y}/{x}',
    
    
    # Topo Map
    tiles='https://server.arcgisonline.com/arcgis/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}',
    attr='CATALYST.Earth',
    location=[44.75, -122.1],   
)

geojson = r"c:\Data_001\SAR_Changes\Demo AOI.geojson"
folium.GeoJson(
    geojson,
    name='geojson'
).add_to(m)

m

In [ ]:
#Import CATALYST Professional API
import pci

#Import Image Ingest Algorithm [SARINGEST]
from pci.saringest import saringest
from pci.exceptions import PCIException

#import Change Detection Algorithm [CCDINTEN]
from pci.ccdinten import ccdinten
from pci.exceptions import PCIException

#import Polygonization of Changes Algorithm [EXPOLRAS]
from pci.expolras import expolras
from pci.exceptions import PCIException

In [ ]:
#Set up ingestion of AOI for 2 Sentinel-1 GRD Images (local files)
ingest1 = "C:\Data_001\SAR_Changes\S1A_IW_GRDH_1SDV_20201204T141406_20201204T141435_035537_0427BA_0B78.SAFE\manifest.safe"
ingest2 = "C:\Data_001\SAR_Changes\S1A_IW_GRDH_1SDV_20201216T141406_20201216T141435_035712_042DD6_D9D5.SAFE\manifest.safe"

#sub section of larger Sentinel-1 image
input_window = [6600, 14000, 6200, 3500] 

imported1 = r"C:\Data_001\SAR_Changes\20201204_sub.pix"
imported2 = r"C:\Data_001\SAR_Changes\20201216_sub.pix"

In [ ]:
# ingest images from local files
#import first image
try:
    saringest(ingest1, imported1, input_window)
except PCIException as e:
    print(e)
except Exception as e:
    print(e)

#import second image
try:
    saringest(ingest2, imported2, input_window)
except PCIException as e:
    print(e)
except Exception as e:
    print(e)

In [ ]:
#alternatively, search and download your own images from Copernicus SciHub
# import sentinelsat library
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt

# import geopandas to format outputs from Scihub search into a table
import geopandas as gpd 

# pass credentials
user = ('pci_user')
password=('PCIGeomatics')

# connect to API
api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

#define search footprint
footprint = geojson_to_wkt(read_geojson('C:\Data_001\SAR_Changes\Demo AOI.geojson'))

products = api.query(footprint,
                     date = ('20201203', '20201218'),
                     producttype=('GRD'),
                     orbitdirection= 'DESCENDING')
api.to_geodataframe(products)


In [ ]:
#download specific images -  copy/paste from table above
# images will be stored in root folder of JN

# Download 20201204 GRD image
# api.download('9f9efee1-bacd-4dc7-a7a0-4c2ab727d6f0')

# Download 20201216 GRD image
# api.download('07b5c32a-722f-464f-b027-b394f62618b8')

# sample calls from sentinelsat api
# query long form metadata
# api.get_product_odata('07b5c32a-722f-464f-b027-b394f62618b8', full=True)


In [ ]:
# set up for CCDINTEN (local files)
# input image and reference images stay the same
ccdinten_result = "C:\Data_001\SAR_Changes\CCD_changes.pix" #new output file

#execute ccdinten
try:
    ccdinten(imported2, imported1,[] ,ccdinten_result, [])
except PCIException as e:
    print(e)
except Exception as e:
    print(e)

In [ ]:
# filter changes and extract polygons
# set up for EXPOLRAS
expolras_result = "C:\Data_001\SAR_Changes\change_polygons.pix" #new output file
threshold_value = [90] #only retain changes above this ranked change percentile value between 0-100
min_contig_pixels = [50] #only keep changes where neighboring pixels are equal to or greater than this value

try:
    expolras(ccdinten_result, [4], '', threshold_value, min_contig_pixels,[], expolras_result, '', '')
except PCIException as e:
    print(e)
except Exception as e:
    print(e)